In [5]:
!pip install openai colorama

In [101]:
!pip install tavily-python

# Objective:

Implement a ReAct (Reasoning + Acting) agent that integrates a Large Language Model (LLM) and a web search tool to research a given topic. The agent will plan its research process, use the web to gather relevant information, and finally compile a structured report.

Given a user-defined topic, the agent should:
1. Generate a list of key research questions using an LLM (Planning Phase).
2. Search the web for answers using a search tool (Acting Phase).
3.  Compile a structured report based on the gathered information.

### ReAct System Prompt

In [1]:
# @title
import json
import re
from dataclasses import dataclass
from typing import Callable


def get_fn_signature(fn: Callable) -> dict:
    """
    Generates the signature for a given function.

    Args:
        fn (Callable): The function whose signature needs to be extracted.

    Returns:
        dict: A dictionary containing the function's name, description,
              and parameter types.
    """
    fn_signature: dict = {
        "name": fn.__name__,
        "description": fn.__doc__,
        "parameters": {"properties": {}},
    }
    schema = {
        k: {"type": v.__name__} for k, v in fn.__annotations__.items() if k != "return"
    }
    fn_signature["parameters"]["properties"] = schema
    return fn_signature


def validate_arguments(tool_call: dict, tool_signature: dict) -> dict:
    """
    Validates and converts arguments in the input dictionary to match the expected types.

    Args:
        tool_call (dict): A dictionary containing the arguments passed to the tool.
        tool_signature (dict): The expected function signature and parameter types.

    Returns:
        dict: The tool call dictionary with the arguments converted to the correct types if necessary.
    """
    properties = tool_signature["parameters"]["properties"]

    # TODO: This is overly simplified but enough for simple Tools.
    type_mapping = {
        "int": int,
        "str": str,
        "bool": bool,
        "float": float,
    }

    for arg_name, arg_value in tool_call["arguments"].items():
        expected_type = properties[arg_name].get("type")

        if not isinstance(arg_value, type_mapping[expected_type]):
            tool_call["arguments"][arg_name] = type_mapping[expected_type](arg_value)

    return tool_call


class Tool:
    """
    A class representing a tool that wraps a callable and its signature.

    Attributes:
        name (str): The name of the tool (function).
        fn (Callable): The function that the tool represents.
        fn_signature (str): JSON string representation of the function's signature.
    """

    def __init__(self, name: str, fn: Callable, fn_signature: str):
        self.name = name
        self.fn = fn
        self.fn_signature = fn_signature

    def __str__(self):
        return self.fn_signature

    def run(self, **kwargs):
        """
        Executes the tool (function) with provided arguments.

        Args:
            **kwargs: Keyword arguments passed to the function.

        Returns:
            The result of the function call.
        """
        return self.fn(**kwargs)


def tool(fn: Callable):
    """
    A decorator that wraps a function into a Tool object.

    Args:
        fn (Callable): The function to be wrapped.

    Returns:
        Tool: A Tool object containing the function, its name, and its signature.
    """

    def wrapper():
        fn_signature = get_fn_signature(fn)
        return Tool(
            name=fn_signature.get("name"), fn=fn, fn_signature=json.dumps(fn_signature)
        )

    return wrapper()


@dataclass
class TagContentResult:
    """
    A data class to represent the result of extracting tag content.

    Attributes:
        content (List[str]): A list of strings containing the content found between the specified tags.
        found (bool): A flag indicating whether any content was found for the given tag.
    """

    content: list[str]
    found: bool


def extract_tag_content(text: str, tag: str) -> TagContentResult:
    """
    Extracts all content enclosed by specified tags (e.g., <thought>, <response>, etc.).

    Parameters:
        text (str): The input string containing multiple potential tags.
        tag (str): The name of the tag to search for (e.g., 'thought', 'response').

    Returns:
        dict: A dictionary with the following keys:
            - 'content' (list): A list of strings containing the content found between the specified tags.
            - 'found' (bool): A flag indicating whether any content was found for the given tag.
    """
    # Build the regex pattern dynamically to find multiple occurrences of the tag
    tag_pattern = rf"<{tag}>(.*?)</{tag}>"

    # Use findall to capture all content between the specified tag
    matched_contents = re.findall(tag_pattern, text, re.DOTALL)

    # Return the dataclass instance with the result
    return TagContentResult(
        content=[content.strip() for content in matched_contents],
        found=bool(matched_contents),
    )


### Import Relevant Dependencies and OpenAI Client

In [123]:
import os
import re
import math
import json
from google.colab import userdata

from openai import OpenAI

client = OpenAI(
    api_key=userdata.get('OPENAI_API_KEY')
)
model = "gpt-4o"

In [122]:
REACT_SYSTEM_PROMPT = """
You are a function-calling AI agent that follows a strict ReAct loop:
Thought → Action → Observation.

You have access to the following tools inside <tools></tools> tags.
You MUST follow the rules carefully.

<tools>
{tools_signatures}
</tools>

=========================================
### HOW YOU MUST REASON AND ACT
=========================================

1. ALWAYS think step-by-step inside <thought></thought> XML tags.
2. When a user provides a topic:
   - FIRST call the `generate_research_questions` tool.
   - THEN, for EACH research question returned, call the
     `research_information_extractor` tool.
3. NEVER guess or fabricate facts. Always rely on tool output.
4. You may call multiple tools in sequence.
5. AFTER all tool calls are completed, provide the final answer inside
   <response></response>.
6. If a question cannot be answered by tools, respond directly using
   <response></response>.

=========================================
### TOOL CALL FORMAT (REQUIRED)
=========================================

Every tool call MUST use EXACTLY this format:

<tool_call>
{{"name": "<tool-name>", "arguments": <python-dict>, "id": <incrementing-int>}}
</tool_call>

Example:

<tool_call>
{{"name": "get_weather", "arguments": {{"city": "Madrid"}}, "id": 0}}
</tool_call>

After a tool is executed, you will receive its output wrapped in:

<observation>{{<id>: <tool-output>}}</observation>

=========================================================
### FINAL ANSWER REQUIREMENT
=========================================================

After finishing ALL tool calls, provide the final analysis or summary inside:

<response> ... </response>
"""


BASE_SYSTEM_PROMPT = ""

### Helper Functions

In [121]:
# @title
"""
This is a collection of helper functions and methods we are going to use in
the Agent implementation.
"""

import re
import time

from colorama import Fore
from colorama import Style

from dataclasses import dataclass


def completions_create(client, messages: list, model: str) -> str:
    """
    Sends a request to the client's `completions.create` method to interact with the language model.

    Args:
        client (OpenAI): The OpenAI client object
        messages (list[dict]): A list of message objects containing chat history for the model.
        model (str): The model to use for generating tool calls and responses.

    Returns:
        str: The content of the model's response.
    """
    response = client.chat.completions.create(messages=messages, model=model)
    return str(response.choices[0].message.content)


def build_prompt_structure(prompt: str, role: str, tag: str = "") -> dict:
    """
    Builds a structured prompt that includes the role and content.

    Args:
        prompt (str): The actual content of the prompt.
        role (str): The role of the speaker (e.g., user, assistant).

    Returns:
        dict: A dictionary representing the structured prompt.
    """
    if tag:
        prompt = f"<{tag}>{prompt}</{tag}>"
    return {"role": role, "content": prompt}

def update_chat_history(history: list, msg: str, role: str):
    """
    Updates the chat history by appending the latest response.

    Args:
        history (list): The list representing the current chat history.
        msg (str): The message to append.
        role (str): The role type (e.g. 'user', 'assistant', 'system')
    """
    history.append(build_prompt_structure(prompt=msg, role=role))


class ChatHistory(list):
    def __init__(self, messages: list | None = None, total_length: int = -1):
        """Initialise the queue with a fixed total length.

        Args:
            messages (list | None): A list of initial messages
            total_length (int): The maximum number of messages the chat history can hold.
        """
        if messages is None:
            messages = []

        super().__init__(messages)
        self.total_length = total_length

    def append(self, msg: str):
        """Add a message to the queue.

        Args:
            msg (str): The message to be added to the queue
        """
        if len(self) == self.total_length:
            self.pop(0)
        super().append(msg)



class FixedFirstChatHistory(ChatHistory):
    def __init__(self, messages: list | None = None, total_length: int = -1):
        """Initialise the queue with a fixed total length.

        Args:
            messages (list | None): A list of initial messages
            total_length (int): The maximum number of messages the chat history can hold.
        """
        super().__init__(messages, total_length)

    def append(self, msg: str):
        """Add a message to the queue. The first messaage will always stay fixed.

        Args:
            msg (str): The message to be added to the queue
        """
        if len(self) == self.total_length:
            self.pop(1)
        super().append(msg)

def fancy_print(message: str) -> None:
    """
    Displays a fancy print message.

    Args:
        message (str): The message to display.
    """
    print(Style.BRIGHT + Fore.CYAN + f"\n{'=' * 50}")
    print(Fore.MAGENTA + f"{message}")
    print(Style.BRIGHT + Fore.CYAN + f"{'=' * 50}\n")
    time.sleep(0.5)


def fancy_step_tracker(step: int, total_steps: int) -> None:
    """
    Displays a fancy step tracker for each iteration of the generation-reflection loop.

    Args:
        step (int): The current step in the loop.
        total_steps (int): The total number of steps in the loop.
    """
    fancy_print(f"STEP {step + 1}/{total_steps}")


@dataclass
class TagContentResult:
    """
    A data class to represent the result of extracting tag content.

    Attributes:
        content (List[str]): A list of strings containing the content found between the specified tags.
        found (bool): A flag indicating whether any content was found for the given tag.
    """

    content: list[str]
    found: bool


def extract_tag_content(text: str, tag: str) -> TagContentResult:
    """
    Extracts all content enclosed by specified tags (e.g., <thought>, <response>, etc.).

    Parameters:
        text (str): The input string containing multiple potential tags.
        tag (str): The name of the tag to search for (e.g., 'thought', 'response').

    Returns:
        dict: A dictionary with the following keys:
            - 'content' (list): A list of strings containing the content found between the specified tags.
            - 'found' (bool): A flag indicating whether any content was found for the given tag.
    """
    # Build the regex pattern dynamically to find multiple occurrences of the tag
    tag_pattern = rf"<{tag}>(.*?)</{tag}>"

    # Use findall to capture all content between the specified tag
    matched_contents = re.findall(tag_pattern, text, re.DOTALL)

    # Return the dataclass instance with the result
    return TagContentResult(
        content=[content.strip() for content in matched_contents],
        found=bool(matched_contents),
    )


### Create Tools

In [120]:
import json
import requests
from tavily import TavilyClient

RESEARCH_QUESTION_PROMPT = """
You are an expert researcher. Your task is given a topic, generate 5-6 well-
structured research and academically rigorous research questions.

Requirements:
- Each questions must cover different aspects of the topic
- Questions must be researchable
- Avoid yes/no questions

Example: If the topic is climate change, generate questions like:
- What are the main causes of climate change?
- How has global temperature changed over the past century?
- What policies are in place for climate change?

Output Format (JSON):
{{
  "topic": "<input-topic>",
  "research_questions": [
    "Question 1...",
    "Questions 2...",
    "Questions 3...",
    "Questions 4...",
    "Questions 5...",
    "Questions 6 optional..."
  ]
}}
Topic: {topic}
"""

RESEARCH_RETRIEVAL_PROMPT = """
You are an expert research analyst. Given a research question and a set of
Google search results, extract ONLY the most relevant, accurate, and recent
information.

Tasks:
1. Read the search snippets carefully.
2. Extract 4–6 key findings that directly answer the question.
3. Prefer recent information (2022–2025).
4. Avoid speculation — only use what the snippets support.
5. Cite sources using: (Source: <domain>)

Your response MUST be valid JSON with this structure (do not include any extra text):

- "research_question": string (the original question)
- "key_points": array of 4–6 short, factual bullet points (strings)
- "sources": array of 2–5 source identifiers (domains or site names)

Research Question:
{rq}

Search Results:
{search_results}
"""

REPORT_PROMPT = """
You are an expert research writer. You will be given:

- A topic
- A list of research entries in JSON form

Each research entry has the following fields:
- "research_question": string
- "key_points": list of short bullet point strings
- "sources": list of source identifiers (e.g., domains or site names)

Your task is to write a clear, well-structured report in MARKDOWN with the following structure:

1. A descriptive, concise TITLE on the first line (starting with "# ").
2. An INTRODUCTION section (1–2 paragraphs) that:
   - Summarizes the overall topic
   - Briefly previews the main themes across the research questions
3. A SECTION for EACH research question, in order:
   - Use "##" or "###" headings (e.g., "## 1. [Question]")
   - Summarize and synthesize the key points into cohesive paragraphs
   - Optionally include bullet lists for clarity
   - Naturally incorporate sources (e.g., "According to NASA and IPCC reports...")
4. A CONCLUSION section that:
   - Highlights patterns and overarching insights across the questions
   - Mentions any limitations or gaps in the findings
   - Suggests possible directions for further research

Style requirements:
- Neutral, academic tone.
- No extra commentary about being an AI.
- Do NOT include the raw JSON in the report.
- Do NOT invent completely new facts; stay anchored to the key points.
- You may infer high-level patterns from the provided points.

Topic:
{topic}

Research data (JSON list of research entries):
{research_data}
"""


@tool
def generate_research_questions(topic: str):
  """
  Generate several research questions related to the given topic.

  This function returns 5-6 well-structured research questions that cover
  different aspects of the topic.

  Args:
      topic (str): A topic that the user needs research questions for
  """
  prompt = RESEARCH_QUESTION_PROMPT.format(topic=topic)

  response = client.chat.completions.create(
      model=model,
      messages=[{"role": "user", "content": prompt}],
      response_format={"type": "json_object"}
  )

  return response.choices[0].message.content

def search_tavily(query: str, n: int = 5):
    client = TavilyClient(api_key=userdata.get("TAVILY_API_KEY"))

    response = client.search(
        query=query,
        max_results=n,
        include_answer=False,
        include_raw_content=False,
    )

    return response



@tool
def get_research_answer(rq: str):
    """
    Given a research question, search the web using Tavily,
    then extract key points using GPT-4o.
    """
    # 1. Run Tavily search
    search_json = search_tavily(query=rq, n=6)

    # 2. Format search results for the LLM
    items = search_json.get("results", [])
    search_results_text = ""

    for item in items:
        title = item.get("title", "")
        snippet = item.get("content", "")
        link = item.get("url", "")
        source = link.split("/")[2] if "://" in link else "unknown"

        search_results_text += (
            f"- Title: {title}\n"
            f"- Snippet: {snippet}\n"
            f"- Source: {source}\n"
            f"- URL: {link}\n\n"
        )

    # 3. Build the prompt
    prompt = RESEARCH_RETRIEVAL_PROMPT.format(
        rq=rq,
        search_results=search_results_text,
    )

    # 4. GPT extraction
    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        response_format={"type": "json_object"},
    )

    return response.choices[0].message.content


@tool
def write_report(topic: str, research_data: str):
  """
  Compile a structured markdown research report fromn multiple get_research_answer
  results.

  Args:
    topic (str): The overall topic of the research paper.
    research_data (str): A JSON list of research entries.

  Returns:
    str: The generated markdown report.
  """
  prompt = REPORT_PROMPT.format(
      topic=topic,
      research_data=research_data
  )

  response = client.chat.completions.create(
      model=model,
      messages=[{"role": "user", "content": prompt}],
  )

  return response.choices[0].message.content

In [124]:
tools_signatures = generate_research_questions.fn_signature + ",\n" + get_research_answer.fn_signature + ",\n" + write_report.fn_signature
print(tools_signatures)

{"name": "generate_research_questions", "description": "\n  Generate several research questions related to the given topic.\n\n  This function returns 5-6 well-structured research questions that cover\n  different aspects of the topic.\n\n  Args:\n      topic (str): A topic that the user needs research questions for\n  ", "parameters": {"properties": {"topic": {"type": "str"}}}},
{"name": "get_research_answer", "description": "\n    Given a research question, search the web using Tavily,\n    then extract key points using GPT-4o.\n    ", "parameters": {"properties": {"rq": {"type": "str"}}}},
{"name": "write_report", "description": "\n  Compile a structured markdown research report fromn multiple get_research_answer\n  results.\n\n  Args:\n    topic (str): The overall topic of the research paper.\n    research_data (str): A JSON list of research entries.\n  \n  Returns:\n    str: The generated markdown report.\n  ", "parameters": {"properties": {"topic": {"type": "str"}, "research_data

### Create Agent Class

In [125]:
class ReactAgent:
    """
    A class that represents an agent using the ReAct logic that interacts with tools to process
    user inputs, make decisions, and execute tool calls. The agent can run interactive sessions,
    collect tool signatures, and process multiple tool calls in a given round of interaction.

    Attributes:
        client (OpenAI): The OpenAI client used to handle model-based completions.
        model (str): The name of the model used for generating responses. Default is "gpt-4o".
        tools (list[Tool]): A list of Tool instances available for execution.
        tools_dict (dict): A dictionary mapping tool names to their corresponding Tool instances.
    """

    def __init__(
        self,
        tools: Tool | list[Tool],
        model: str = "gpt-4o",
        system_prompt: str = BASE_SYSTEM_PROMPT,
    ) -> None:
        self.client = OpenAI(
            api_key=userdata.get('OPENAI_API_KEY')
        )
        self.model = model
        self.system_prompt = system_prompt
        self.tools = tools if isinstance(tools, list) else [tools]
        self.tools_dict = {tool.name: tool for tool in self.tools}

        if self.tools:
          tools_sig = self.add_tool_signatures()
          self.system_prompt = (
              self.system_prompt
              + "\n"
              + REACT_SYSTEM_PROMPT.format(tools_signatures=tools_sig)
          )
        else:
          self.system_prompt = system_prompt

    def add_tool_signatures(self) -> str:
        """
        Collects the function signatures of all available tools.

        Returns:
            str: A concatenated string of all tool function signatures in JSON format.
        """
        return "".join([tool.fn_signature for tool in self.tools])

    def process_tool_calls(self, tool_calls_content: list) -> dict:
        """
        Processes each tool call, validates arguments, executes the tools, and collects results.

        Args:
            tool_calls_content (list): List of strings, each representing a tool call in JSON format.

        Returns:
            dict: A dictionary where the keys are tool call IDs and values are the results from the tools.
        """
        observations = {}
        for tool_call_str in tool_calls_content:
            tool_call = json.loads(tool_call_str)
            tool_name = tool_call["name"]
            tool = self.tools_dict[tool_name]

            print(Fore.GREEN + f"\nUsing Tool: {tool_name}")

            # Validate and execute the tool call
            validated_tool_call = validate_arguments(
                tool_call, json.loads(tool.fn_signature)
            )
            print(Fore.GREEN + f"\nTool call dict: \n{validated_tool_call}")

            result = tool.run(**validated_tool_call["arguments"])
            print(Fore.GREEN + f"\nTool result: \n{result}")

            # Store the result using the tool call ID
            observations[validated_tool_call["id"]] = result

        return observations

    def run(
        self,
        user_msg: str,
        max_rounds: int = 10,
    ) -> str:
        """
        Executes a user interaction session, where the agent processes user input, generates responses,
        handles tool calls, and updates chat history until a final response is ready or the maximum
        number of rounds is reached.

        Args:
            user_msg (str): The user's input message to start the interaction.
            max_rounds (int, optional): Maximum number of interaction rounds the agent should perform. Default is 10.

        Returns:
            str: The final response generated by the agent after processing user input and any tool calls.
        """
        user_prompt = build_prompt_structure(
            prompt=user_msg, role="user", tag="question"
        )
        if self.tools:
            self.system_prompt += (
                self.system_prompt
                + "\n"
                + REACT_SYSTEM_PROMPT.format(
                    tools_signatures=self.add_tool_signatures()
                )
            )

        chat_history = ChatHistory(
            [
                build_prompt_structure(
                    prompt=self.system_prompt,
                    role="system",
                ),
                user_prompt,
            ]
        )

        if self.tools:
            # Run the ReAct loop for max_rounds
            for _ in range(max_rounds):

                completion = completions_create(self.client, chat_history, self.model)

                response = extract_tag_content(str(completion), "response")
                if response.found:
                    return response.content[0]

                thought = extract_tag_content(str(completion), "thought")
                tool_calls = extract_tag_content(str(completion), "tool_call")

                update_chat_history(chat_history, completion, "assistant")

                print(Fore.MAGENTA + f"\nThought: {thought.content[0]}")

                if tool_calls.found:
                    observations = self.process_tool_calls(tool_calls.content)
                    print(Fore.BLUE + f"\nObservations: {observations}")
                    update_chat_history(chat_history, f"{observations}", "user")

        return completions_create(self.client, chat_history, self.model)

In [126]:
agent = ReactAgent(
    tools=[generate_research_questions, get_research_answer, write_report],
    model="gpt-4o",
)

In [127]:
agent.run(user_msg="Climate Change")


Thought: To begin researching the topic of Climate Change, the first step will be to generate relevant research questions that cover different aspects of this topic.

Using Tool: generate_research_questions

Tool call dict: 
{'name': 'generate_research_questions', 'arguments': {'topic': 'Climate Change'}, 'id': 0}

Tool result: 
{
  "topic": "Climate Change",
  "research_questions": [
    "What are the main causes of climate change?",
    "How has the global temperature changed over the past century?",
    "What policies are currently in place to address climate change, and how effective are they?",
    "What are the impacts of climate change on biodiversity and ecosystems?",
    "How does climate change affect natural disasters and their frequency or severity?",
    "What role do renewable energy sources play in mitigating climate change?"
  ]
}

Observations: {0: '{\n  "topic": "Climate Change",\n  "research_questions": [\n    "What are the main causes of climate change?",\n    "How

"# Understanding and Addressing Climate Change\n\n## Introduction\n\nClimate change is an increasingly pressing global concern characterized by the long-term alteration of temperature and typical weather patterns in Earth's atmosphere. Triggered largely by human activities, this phenomenon poses significant challenges to natural ecosystems and human society. This report examines six key aspects of climate change, including its causes, historical temperature changes, policy responses, impacts on biodiversity and natural disasters, and the role of renewable energy. The insights provided highlight the complexity and interconnectedness of factors contributing to and mitigating climate change.\n\n## 1. What are the main causes of climate change?\n\nThe primary driver behind climate change is the greenhouse effect, where greenhouse gases trap heat from the sun in Earth's atmosphere, leading to global warming (climate.ec.europa.eu). Human activities, notably the burning of fossil fuels such a

### Full Generated Report

# Understanding and Addressing Climate Change

## Introduction

Climate change is an increasingly pressing global concern characterized by the long-term alteration of temperature and typical weather patterns in Earth's atmosphere. Triggered largely by human activities, this phenomenon poses significant challenges to natural ecosystems and human society. This report examines six key aspects of climate change, including its causes, historical temperature changes, policy responses, impacts on biodiversity and natural disasters, and the role of renewable energy. The insights provided highlight the complexity and interconnectedness of factors contributing to and mitigating climate change.

## 1. What are the main causes of climate change?

The primary driver behind climate change is the greenhouse effect, where greenhouse gases trap heat from the sun in Earth's atmosphere, leading to global warming (climate.ec.europa.eu). Human activities, notably the burning of fossil fuels such as coal, oil, and gas, are the most significant contributors, accounting for approximately 68% of global greenhouse gas emissions (un.org). Activities like deforestation and wetland degradation further intensify the concentration of these gases (nrdc.org). Methane, alongside carbon dioxide, plays a crucial role in exacerbating global warming, particularly in the short term (climate.ec.europa.eu). While natural processes such as volcanic activity contribute to climate change, their emissions are significantly lower compared to anthropogenic sources (epa.gov).

## 2. How has the global temperature changed over the past century?

The past century has witnessed a noticeable increase in the Earth's average global temperature, rising by approximately 0.8 degrees Celsius (1.4 degrees Fahrenheit) (theworldcounts.com). The global surface temperature increased by about 0.6 degrees Celsius during the 20th century (archive.ipcc.ch). Since 1901, this temperature rise has occurred at an average rate of 0.17 degrees Fahrenheit per decade (epa.gov). From a broader historical perspective, the average global temperature has increased by at least 1.1 degrees Celsius (1.9 degrees Fahrenheit) since 1880 (earthobservatory.nasa.gov). Since 1975, combined land and ocean temperatures have been warming at an average rate of 0.36 degrees Fahrenheit (0.20 degrees Celsius) per decade (climate.gov).

## 3. What policies are currently in place to address climate change, and how effective are they?

A variety of policies have been implemented worldwide to combat climate change, with an emphasis on real-world results and necessary compromises. In the United States, individual states are adopting measures such as carbon pricing, emission limits, renewable portfolio standards, and promoting cleaner transportation (c2es.org). Globally, countries are developing policies that reconcile environmental goals with economic realities (worldbank.org). Some U.S. states are committed to reducing greenhouse gas emissions through carbon markets, clean energy standards, and investments, independent of federal policy (css.umich.edu). A significant legislative effort is represented by the Inflation Reduction Act in the U.S., which aims to significantly reduce emissions (2021-2025.state.gov).

## 4. What are the impacts of climate change on biodiversity and ecosystems?

Climate change has dramatically increased the frequency and intensity of extreme weather events, adversely affecting biodiversity (royalsociety.org). Shifts in climate patterns disrupt natural cycles, posing extinction threats to certain species (greenpeace.org.uk). Changes in temperature and precipitation exacerbate pressures on species, intensifying their vulnerability to human activities (royalsociety.org). Additionally, climate change facilitates the expansion of invasive species, which compete with native species, disrupting local ecosystems (greenpeace.org.uk). Nonetheless, biodiversity in forests and peatlands plays a vital role in combating climate change by acting as carbon sinks (source.washu.edu).

## 5. How does climate change affect natural disasters and their frequency or severity?

The influence of climate change on natural disasters is evident in the increased frequency and severity of events such as heatwaves, droughts, wildfires, and floods (bbc.com). Rising temperatures augment evaporation rates, contributing to more frequent and intense rainfall events, which can trigger landslides and floods (pbs.org). Furthermore, higher global temperatures have the potential to increase the wind speeds of tropical storms (earthobservatory.nasa.gov). The United States has seen an increase in billion-dollar disasters over the past decade, with climate change being a key factor in their rising frequency and intensity (climate.gov). Sea-level rise and shifts in water availability further exacerbate the occurrence and severity of natural disasters like floods and hurricanes (usgs.gov).

## 6. What role do renewable energy sources play in mitigating climate change?

Renewable energy sources are crucial in alleviating climate change by significantly reducing carbon emissions when replacing fossil fuels (researchgate.net). They promote sustainable development by reducing greenhouse gas emissions and enhancing energy efficiency (journalspub.com). Solar and wind power, in particular, are standout sources as they produce electricity without emitting carbon, thus bolstering air quality and mitigating climate impacts (energydigital.com). The transition to renewable energy technologies aids in lowering greenhouse gas emissions and provides adaptive strategies to counteract climate change's effects on local communities (sciencedirect.com).

## Conclusion

The analysis of the various aspects of climate change highlights a few overarching themes. Human activities, particularly fossil fuel combustion and land use changes, are principal drivers of climate change. In response, renewable energy and policy measures play pivotal roles in mitigation efforts. As the global temperature continues to rise, it will likely exacerbate natural disasters and disrupt biodiversity. However, there exist potential solutions in the form of renewable energy adoption and targeted policy implementations. Notable gaps remain in understanding fully the long-term implications of natural processes on climate change and the efficacy of global policy responses. Future research should focus on developing and implementing globally coherent strategies that include adaptive measures to enhance resilience against climate impacts.

### Final Report
The system I made uses an LLM (GPT-4o) to not only answer given queries, but also to use ReAct pattern in order to determine the steps and which tool to use
to solve a given problem.

The LLM generates an internal thought where it decides what to do next. Based on
that thought, it decides which of the tools (if any) should be used and also what arguments need to be passed for that tool.

Finally, after recieving the output from performing the action, it updates its internal understanding and goes back to the first step where it deciedes what to do again, creating a thought ▶ action ▶ observation loop.

---

The main components of the code are
1. Tool functions
2. Helper functions
3. ReactAgent Class
4. React system prompt and loop

The code has a specific react prompt that the LLM uses to start processing the user query.